In [90]:
import requests
import pandas as pd
from urllib.request import urlopen
import json
import logging
from typing import List
from io import StringIO

In [91]:
logging.basicConfig(filename='esg_data_fetch.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [92]:
class ESGDataFetcher:
    """
    A class to fetch and store ESG data for companies listed in the S&P 500.

    Attributes:
    -----------
    sp500_tickers : List[str]
        A list of ticker symbols for companies in the S&P 500.
    
    Methods:
    --------
    fetch_sp500_tickers() -> List[str]:
        Fetches the list of S&P 500 tickers from Wikipedia.
    
    get_latest_esg_data(ticker: str) -> pd.DataFrame:
        Fetches the latest ESG data for a given ticker from Yahoo Finance.
    
    collect_esg_data() -> pd.DataFrame:
        Fetches and saves the latest ESG data for all S&P 500 companies.
    """

    def __init__(self):
        self.sp500_tickers = self.fetch_sp500_tickers()

    def fetch_sp500_tickers(self) -> List[str]:
        """
        Fetches the list of S&P 500 ticker symbols from Wikipedia.
        
        Returns:
        --------
        List[str]:
            A list of ticker symbols for companies in the S&P 500.
        """
        wikipedia_page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
        
        # Wrap the HTML content in a StringIO object
        sp500_tables = pd.read_html(StringIO(wikipedia_page))
        ticker_dataframe = sp500_tables[0]
        return ticker_dataframe['Symbol'].tolist()

    def get_latest_esg_data(self, ticker: str) -> pd.DataFrame:
        """
        Fetches the latest ESG data for a given ticker from Yahoo Finance.
        
        Parameters:
        -----------
        ticker : str
            The ticker symbol of the company.
        
        Returns:
        --------
        pd.DataFrame:
            A DataFrame containing the latest ESG data for the given ticker.
        """
        esg_url = f"https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={ticker}"
        connection = urlopen(esg_url)
        raw_data = connection.read()
        json_data = json.loads(raw_data)
        
        # Check if the expected data structure is present
        if "result" in json_data["esgChart"] and len(json_data["esgChart"]["result"]) > 0:
            if "symbolSeries" in json_data["esgChart"]["result"][0]:
                esg_series_data = json_data["esgChart"]["result"][0]["symbolSeries"]
                esg_dataframe = pd.DataFrame(esg_series_data)
                esg_dataframe["timestamp"] = pd.to_datetime(esg_dataframe["timestamp"], unit="s")
                
                # Log successful data retrieval
                logging.info(f"Successfully fetched ESG data for ticker: {ticker}")
                
                # Return only the last row
                return esg_dataframe.iloc[-1].to_frame().T
            else:
                error_message = f"'symbolSeries' not found for ticker {ticker}"
                logging.warning(error_message)
                raise KeyError(error_message)
        else:
            error_message = f"No ESG data found for ticker {ticker}"
            logging.warning(error_message)
            raise ValueError(error_message)

    def collect_esg_data(self) -> pd.DataFrame:
        """
        Fetches and saves the latest ESG data for all S&P 500 companies.
        
        Returns:
        --------
        pd.DataFrame:
            A DataFrame containing the latest ESG data for all companies in the S&P 500.
        """
        final_esg_data = []
        success_count = 0
        failure_count = 0
        
        for ticker in self.sp500_tickers:
            try:
                latest_esg_record = self.get_latest_esg_data(ticker)
                latest_esg_record["ticker"] = ticker
                final_esg_data.append(latest_esg_record)
                success_count += 1
            except Exception as e:
                logging.error(f"Failed to fetch data for {ticker}: {e}")
                failure_count += 1
        
        # Print summary
        total_tickers = len(self.sp500_tickers)
        print(f"Processed {total_tickers} stocks: {success_count} with ESG data, {failure_count} without ESG data.")
        
        # Log summary
        logging.info(f"Processed {total_tickers} stocks: {success_count} with ESG data, {failure_count} without ESG data.")
        
        return pd.concat(final_esg_data, ignore_index=True)

In [93]:
esg_fetcher = ESGDataFetcher()
latest_esg_data = esg_fetcher.collect_esg_data()
latest_esg_data.to_csv('latest_esg_data.csv', index=False)

Processed 503 stocks: 434 with ESG data, 69 without ESG data.


In [94]:
latest_esg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   timestamp         434 non-null    object
 1   esgScore          434 non-null    object
 2   governanceScore   434 non-null    object
 3   environmentScore  434 non-null    object
 4   socialScore       434 non-null    object
 5   ticker            434 non-null    object
dtypes: object(6)
memory usage: 20.5+ KB


In [97]:
latest_esg_data['timestamp'] = pd.to_datetime(latest_esg_data['timestamp'])
score_columns = ['esgScore', 'governanceScore', 'environmentScore', 'socialScore']
latest_esg_data[score_columns] = latest_esg_data[score_columns].astype(float)
latest_esg_data['ticker'] = latest_esg_data['ticker'].astype(str)
latest_esg_data.set_index('timestamp', inplace=True)

KeyError: 'timestamp'

In [98]:
latest_esg_data

,esgScore,governanceScore,environmentScore,socialScore,ticker
timestamp,,,,,
2023-09-01,37.35,6.54,17.22,13.59,MMM
2023-09-01,25.44,6.42,7.15,11.87,AOS
2023-09-01,24.83,8.33,2.27,14.24,ABT
2023-09-01,29.93,10.36,2.38,17.19,ABBV
2023-09-01,9.84,4.41,0.82,4.61,ACN
...,...,...,...,...,...
2023-09-01,26.45,4.49,12.70,9.26,XEL
2023-09-01,18.13,5.19,4.27,8.66,XYL
2023-09-01,20.10,4.12,4.53,11.45,YUM


In [99]:
latest_esg_data.columns

Index(['esgScore', 'governanceScore', 'environmentScore', 'socialScore',
       'ticker'],
      dtype='object')

In [100]:
latest_esg_data.describe()

,esgScore,governanceScore,environmentScore,socialScore
count,434.000000,434.000000,434.000000,434.000000
mean,21.962465,7.256429,6.342581,9.599401
std,7.635302,5.398274,7.324519,5.930090
min,7.080000,2.960000,0.000000,0.760000
25%,16.500000,5.260000,1.790000,6.672500
50%,21.240000,6.130000,4.130000,8.955000
75%,26.115000,7.830000,9.155000,11.442500
max,64.510000,62.100000,63.400000,62.860000


In [110]:
latest_esg_data.quantile(.95, method="table", interpolation="nearest")

esgScore            34.76
governanceScore       4.7
environmentScore    16.09
socialScore         13.97
ticker                FTV
Name: 0.95, dtype: object

In [112]:
latest_esg_data[latest_esg_data.esgScore>34.76]

,esgScore,governanceScore,environmentScore,socialScore,ticker
timestamp,,,,,
2023-09-01,37.35,6.54,17.22,13.59,MMM
2021-03-01,38.81,7.96,21.98,8.88,APA
2018-10-01,55.17,48.85,56.89,56.41,TECH
2023-09-01,39.60,8.29,8.84,22.48,BA
2023-09-01,36.17,8.06,10.43,17.69,CAT
2023-09-01,36.60,10.42,17.01,9.18,CVX
2023-09-01,36.07,7.47,20.26,8.34,CTRA
2018-10-01,49.33,48.70,49.44,51.49,ELV
2023-09-01,35.69,7.95,17.66,10.08,EQT
